<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao33bis_Busca_HNSWlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Colab: Introdução ao HNSWlib

Este notebook demonstra o uso da biblioteca `hnswlib` para busca de similaridade (Approximate Nearest Neighbor - ANN).

O HNSW (Hierarchical Navigable Small World) é um algoritmo baseado em grafos que oferece buscas extremamente rápidas com alta precisão.

Vamos:
1.  Instalar as bibliotecas.
2.  Usar o mesmo corpus e modelo de embedding do exemplo anterior (`sentence-transformers`).
3.  Configurar e inicializar um índice HNSW.
4.  Adicionar os dados ao índice.
5.  Realizar uma busca de similaridade.
6.  (Bônus) Salvar e carregar o índice.

### Célula 1: Instalação

Vamos instalar o `hnswlib` e o `sentence-transformers`.

In [1]:
!pip install hnswlib sentence-transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528141 sha256=bab4f435b239acd4dcfaa03d035045685b46461deec73879061ea83df69acf54
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built hnswlib


### Célula 2: Importações e Definição do Modelo

In [2]:
import hnswlib
import numpy as np
from sentence_transformers import SentenceTransformer

# Carrega o mesmo modelo para comparação
model = SentenceTransformer('all-MiniLM-L6-v2')

print("Modelo e bibliotecas carregados.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo e bibliotecas carregados.


### Célula 3: Criação do Corpus e Geração dos Embeddings

Usaremos os mesmos dados do exemplo do FAISS.

In [3]:
# 1. Nosso corpus de documentos
corpus = [
    "O cachorro late alto no quintal.",
    "Gatos são animais muito independentes.",
    "A inteligência artificial está transformando o mundo rapidamente.",
    "FAISS é uma biblioteca otimizada para busca de vetores.",
    "Eu gosto de programar em Python e construir aplicações de IA.",
    "O céu está azul e o dia está ensolarado hoje.",
    "Como funciona a busca por similaridade em larga escala?",
    "Um cão é o melhor amigo do homem."
]

# 2. Gerar os embeddings (vetores)
corpus_embeddings = model.encode(corpus)

# 3. Verificar a dimensionalidade e tipo
d = corpus_embeddings.shape[1]
num_elements = corpus_embeddings.shape[0]

print(f"Número de frases no corpus (num_elements): {num_elements}")
print(f"Dimensionalidade dos embeddings (d): {d}")

# HNSWlib, assim como FAISS, prefere float32
corpus_embeddings = corpus_embeddings.astype('float32')

Número de frases no corpus (num_elements): 8
Dimensionalidade dos embeddings (d): 384


### Célula 4: Inicialização do Índice HNSW

Aqui, a configuração é um pouco diferente do FAISS.

1.  Primeiro, declaramos o espaço de busca (métrica) e a dimensão.
2.  Depois, inicializamos o índice com parâmetros de construção.

Para embeddings de sentenças, o espaço `cosine` (similaridade de cosseno) geralmente funciona melhor que o `l2` (distância Euclidiana).

In [4]:
# 1. Declarar o espaço de busca
# 'l2' para distância Euclidiana
# 'ip' para produto interno (inner product)
# 'cosine' para similaridade de cosseno
space = 'cosine'
index_hnsw = hnswlib.Index(space=space, dim=d)

# 2. Inicializar o índice
# max_elements: O número máximo de elementos que o índice pode conter.
# M: O número de conexões (arestas) por nó no grafo. 16 é um bom padrão.
# ef_construction: Parâmetro que controla o trade-off de velocidade/precisão na *construção* do índice.
index_hnsw.init_index(max_elements=num_elements, ef_construction=200, M=16)

# 3. (Opcional) Permitir substituição de elementos (não necessário aqui)
# index_hnsw.set_replace_deleted(True)

print("Índice HNSW inicializado.")

Índice HNSW inicializado.


### Célula 5: Adicionando Dados ao Índice

Precisamos de duas coisas para adicionar dados:
1.  Os vetores (embeddings).
2.  Um array de IDs (rótulos) para cada vetor. Usaremos simplesmente 0, 1, 2...

In [5]:
# 1. Criar os IDs para nossos elementos
# Deve ser um array de inteiros
labels_ids = np.arange(num_elements)

# 2. Adicionar os vetores e seus IDs ao índice
index_hnsw.add_items(corpus_embeddings, labels_ids)

# 3. Verificar quantos itens estão no índice
print(f"Total de vetores no índice: {index_hnsw.get_current_count()}")

Total de vetores no índice: 8


### Célula 6: Realizando a Busca (Query)

A busca no HNSWlib tem um parâmetro chave: `ef`.

* `ef` (search-time `ef`): Controla o trade-off de velocidade/precisão na *busca*. Deve ser maior ou igual a `k` (o número de vizinhos que você quer).
* Quanto maior o `ef`, mais precisa a busca, mas mais lenta.

In [6]:
# 1. Definir o parâmetro de busca 'ef'
# Para um dataset tão pequeno, 10 é suficiente. Em datasets reais, valores como 50, 100, 200 são comuns.
index_hnsw.set_ef(10)

# 2. Definir a consulta (query)
query_texto = "Quem é o melhor amigo de uma pessoa?"
query_vector = model.encode([query_texto]).astype('float32')

# 3. Definir quantos vizinhos (k) queremos
k = 3

# 4. Realizar a busca
# Retorna (IDs, Distâncias)
labels, distances = index_hnsw.knn_query(query_vector, k=k)

# 5. Mostrar os resultados
print(f"--- Resultados para a query: '{query_texto}' ---")
print("\nIDs (Labels):", labels)
print("Distâncias (Cosine):", distances)
print("(Nota: Distância Cosseno = 1 - Similaridade Cosseno. Menor é melhor.)")

print("\nFrases mais similares:")
# Note que `labels` é um array 2D, por isso usamos labels[0]
for i, (idx, dist) in enumerate(zip(labels[0], distances[0])):
    print(f"{i+1}. Frase: '{corpus[idx]}'")
    print(f"   Distância (Cosine): {dist:.4f}")

--- Resultados para a query: 'Quem é o melhor amigo de uma pessoa?' ---

IDs (Labels): [[7 5 1]]
Distâncias (Cosine): [[0.27975243 0.43291384 0.5042811 ]]
(Nota: Distância Cosseno = 1 - Similaridade Cosseno. Menor é melhor.)

Frases mais similares:
1. Frase: 'Um cão é o melhor amigo do homem.'
   Distância (Cosine): 0.2798
2. Frase: 'O céu está azul e o dia está ensolarado hoje.'
   Distância (Cosine): 0.4329
3. Frase: 'Gatos são animais muito independentes.'
   Distância (Cosine): 0.5043


### Célula 7: (Bônus) Salvando e Carregando o Índice

O `hnswlib` torna muito fácil salvar e carregar um índice pré-construído.

In [8]:
import os

# 1. Salvar o índice em um arquivo
index_path = 'meu_indice.bin'
index_hnsw.save_index(index_path)

print(f"Índice salvo em {index_path}")

# --- Agora, vamos carregar e testar ---

# 2. Criar um novo índice "vazio"
index_novo = hnswlib.Index(space=space, dim=d)

# 3. Carregar os dados do arquivo
index_novo.load_index(index_path, max_elements=num_elements)

print("\nÍndice carregado.")

# 4. Testar a busca no índice carregado
index_novo.set_ef(10)
labels_novo, dist_novo = index_novo.knn_query(query_vector, k=k)

print("\n--- Teste de busca no índice carregado ---")
print("IDs (Labels):", labels_novo)
print("Frases mais similares:")
for idx in labels_novo[0]:
    print(f"  - {corpus[idx]}")

# Limpar o arquivo
os.remove(index_path)

Índice salvo em meu_indice.bin

Índice carregado.

--- Teste de busca no índice carregado ---
IDs (Labels): [[7 5 1]]
Frases mais similares:
  - Um cão é o melhor amigo do homem.
  - O céu está azul e o dia está ensolarado hoje.
  - Gatos são animais muito independentes.
